In [ ]:
!nvidia-smi

In [ ]:
#@title WANDB Login { run: "auto", display-mode: "form" }
wandbapi = "" #@param {type:"string"}
!pip install wandb > /dev/null
!wandb login "$wandbapi"
%env USE_WANDB=1

In [ ]:
%cd /content/INFOMCV/
!git pull origin master

In [ ]:
!pip install -r requirements.txt > /dev/null

In [ ]:
#@title Load cached dataset from google drive
drive_project_root = "MyDrive/Colab2/INFOCV" #@param {type:"string"}
dataset_name = "tvhi_augmented.tar.gz" #@param {type:"string"}
import os
from google.colab import drive
drive.mount('/content/drive')

drive_dataset_dir = os.path.join('/content/drive', drive_project_root)
drive_dataset_path = os.path.join('/content/drive', drive_project_root, dataset_name)

!mkdir -p $drive_dataset_dir
!test -e $drive_dataset_path \
  && echo 'Extracting the cached dataset' \
  && cp $drive_dataset_path $dataset_name \
  && tar -xzf $dataset_name

In [ ]:
# Download the datasets
!test ! -e $drive_dataset_path \
  && bash datasets.sh > /dev/null

In [ ]:
# Preprocess the dataset and bake the augmentation variant
!test ! -e $drive_dataset_path \
  && python preprocess.py --process_tvhi=True \
  && python bake.py --augment_repeat_count=25 --bake_input=datasets/TV-HI/data --bake_output=datasets/TV-HI/baked --bake_optical_flow=True

In [ ]:
#@title Cache the processed dataset { display-mode: "form" }
dataset_dir = "datasets/TV-HI/baked*" #@param {type:"string"}
!test ! -e $drive_dataset_path \
  && echo 'Compressing the dataset' \
  && GZIP=-9 tar --totals=USR1 -czf $dataset_name $dataset_dir \
  && echo 'Copying cache to drive' \
  && cp $dataset_name $drive_dataset_path

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_of.py --run_name='of_tvhi' --batch_size=128 --model_name='frames_stanford_v2'

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_of.py --run_name='of_tvhi' --batch_size=128 --model_name='frames_stanford_v2' --lr_step_size=500

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_of.py --run_name='of_tvhi' --batch_size=128 --model_name='frames_stanford_v2' --lr_step_size=600

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_of.py --run_name='of_tvhi' --batch_size=128 --model_name='frames_stanford_v2' --lr_step_size=1000

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_of.py --run_name='of_tvhi' --batch_size=128 --model_name='frames_stanford_v2' --lr_step_size=400 --seed=1337

In [ ]:
# %env PROFILE_TF=1
%env WANDB_START_METHOD=thread
!python train_of.py --run_name='of_tvhi' --batch_size=128 --model_name='frames_stanford_v2' --lr_step_size=300 --seed=1337

In [ ]:
!python train_fusion.py --run_name='fusion_tvhi' --batch_size=128 --model_name='fusion_b67' \
  --fusion_frames_weights=snapshots/model_frames_tvhi_frames_stanford_v2.h5 \
  --fusion_of_weights=snapshots/model_of_tvhi_frames_stanford_v2.h5

In [ ]:
!python train_fusion.py --run_name='fusion_tvhi' --batch_size=128 --model_name='fusion_layerless' \
  --fusion_frames_weights=snapshots/model_frames_tvhi_frames_stanford_v2.h5 \
  --fusion_of_weights=snapshots/model_of_tvhi_frames_stanford_v2.h5 \
  --lr_step_size=500

In [ ]:
!python train_fusion.py --run_name='fusion_tvhi' --batch_size=128 --model_name='fusion_layerless' \
  --fusion_frames_weights=snapshots/model_frames_tvhi_frames_stanford_v2.h5 \
  --fusion_of_weights=snapshots/model_of_tvhi_frames_stanford_v2.h5 \
  --lr_step_size=300

In [ ]:
!python train_fusion.py --run_name='fusion_tvhi' --batch_size=128 --model_name='fusion_b67' \
  --fusion_frames_model_name=frames_stanford_half_v2 \
  --fusion_frames_weights=snapshots/model_frames_tvhi_frames_stanford_v2.h5 \
  --fusion_of_model_name=frames_stanford_half_v2 \
  --fusion_of_weights=snapshots/model_of_tvhi_frames_stanford_v2.h5 \
  --lr_step_size=300

In [ ]:
!python train_fusion.py --run_name='fusion_tvhi' --batch_size=128 --model_name='fusion_layerless' \
  --fusion_frames_model_name=frames_stanford_half_v2 \
  --fusion_frames_weights=snapshots/model_frames_tvhi_frames_stanford_v2.h5 \
  --fusion_of_model_name=frames_stanford_half_v2 \
  --fusion_of_weights=snapshots/model_of_tvhi_frames_stanford_v2.h5 \
  --lr_step_size=300

In [ ]:
!python train_fusion.py --run_name='fusion_tvhi' --batch_size=128 --model_name='fusion_layerless' \
  --fusion_frames_weights=snapshots/model_frames_tvhi_frames_stanford_v2.h5 \
  --fusion_of_weights=snapshots/model_of_tvhi_frames_stanford_v2.h5 \
  --lr_step_size=400